In [1]:
import pandas as pd

# CSV 파일 로드
file_path = './외국음식전문점.csv'
data = pd.read_csv(file_path)

# 데이터프레임의 기본 정보 확인
print(data.info())
print(data.describe())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53319 entries, 0 to 53318
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   store_name  53319 non-null  object
 1   content     51124 non-null  object
 2   date        53314 non-null  object
 3   revisit     53314 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB
None
       store_name content    date revisit
count       53319   51124   53314   53314
unique        737   42580    1527      88
top       타이인플레이트       굿  8.17.토  1번째 방문
freq          200    2092     388   42863
  store_name                                            content    date  \
0  드렁킨타이 목동점                         언제나줄서야먹을수있는테이블좀 늘려주세요 아님확장  7.19.금   
1  드렁킨타이 목동점  목동에서 젤좋아하는 곳인데 오늘 고기도 여러점인데도  너무질겨서 하나도 못먹었어요....   7.4.목   
2  드렁킨타이 목동점        ㆍ드렁킨타이 (태국음식전문)ㆍ소고기쌀국수  10,500팟타이ㆍ똠양꿍ㆍ쏨땀 등등  6.20.목   
3  드렁킨타이 목동점  와~~맛있고, 양이 넉넉합니다.테이블이 많지 않지만 타이밍 잘 맞으면 대기 없이 먹...  6.20.목   
4  드렁킨타이 목동점  친구동네 갔다 

In [3]:
import re
from konlpy.tag import Mecab

# Mecab 형태소 분석기 사용 (필요시 다른 분석기 사용 가능)
mecab = Mecab()

def text_preprocessing(text):
    # 특수문자 제거
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\s]", "", text)
    # 형태소 분석
    tokens = mecab.morphs(text)
    # 토큰을 다시 문자열로 결합
    return " ".join(tokens)

# 텍스트 열에 전처리 적용 (예: 'text_column'이 텍스트 데이터가 있는 열이라고 가정)
data['processed_text'] = data['text_column'].apply(text_preprocessing)

# 전처리된 데이터 확인
print(data['processed_text'].head())

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:
# 필요한 라이브러리 설치 및 임포트
# pip install torch transformers git+https://github.com/SKT-AI/KoBART#egg=kobart rouge

import pandas as pd
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, AdamW
from torch.utils.data import DataLoader, TensorDataset
from rouge import Rouge

# 1. 데이터 로드 및 전처리
# CSV 파일을 로드합니다. 이 단계에서는 데이터가 이미 전처리된 상태라고 가정합니다.
data = pd.read_csv('./외국음식전문점.csv')  # 이전에 전처리된 데이터를 저장한 파일
texts = data['processed_text'].tolist()  # 전처리된 텍스트 열을 리스트로 변환

# 2. KoBART 모델 및 토크나이저 로드
# KoBART는 한국어 문서 요약에 특화된 BART 모델입니다.
# 장점: 한국어에 최적화된 모델로, 문맥을 이해하여 자연스러운 요약을 생성할 수 있습니다.
# 단점: 학습에 많은 자원이 필요하며, 장문을 요약할 때 성능이 제한적일 수 있습니다.
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

# 3. 데이터셋 토큰화
# 텍스트 데이터를 모델에 입력할 수 있도록 토큰화합니다.
inputs = tokenizer(texts, max_length=512, truncation=True, return_tensors='pt', padding=True)

# 예시로 요약할 문장을 동일하게 생성합니다.
summary_texts = ["이 문서는 요약을 필요로 합니다."] * len(texts)
labels = tokenizer(summary_texts, max_length=128, truncation=True, return_tensors='pt', padding=True).input_ids

# 4. 데이터셋 및 데이터로더 구성
# 데이터셋을 구성하고, 모델 학습을 위한 데이터로더를 만듭니다.
# 장점: DataLoader를 통해 배치 학습을 수행할 수 있으며, GPU 메모리 사용을 효율화할 수 있습니다.
# 단점: 데이터가 너무 크면 메모리 부족이 발생할 수 있습니다.
dataset = TensorDataset(inputs.input_ids, labels)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# 5. 옵티마이저 설정
# AdamW 옵티마이저를 사용하여 모델을 학습합니다.
# 장점: AdamW는 일반적인 Adam보다 L2 정규화가 추가되어 과적합을 줄이는 데 도움을 줍니다.
# 단점: 학습률 설정이 민감하여 적절한 값을 찾는 데 시간이 걸릴 수 있습니다.
optimizer = AdamW(model.parameters(), lr=5e-5)

# 6. 모델 학습
# 모델을 학습시키는 과정입니다. 에포크 수를 설정하고, 각 배치마다 손실을 계산하여 역전파합니다.
# 장점: 모델이 학습을 통해 점점 더 나은 예측을 할 수 있게 됩니다.
# 단점: 학습 시간이 오래 걸릴 수 있으며, 특히 대규모 데이터셋의 경우 GPU 자원이 필수적입니다.
model.train()
epochs = 3
for epoch in range(epochs):
    for batch in dataloader:
        input_ids, labels = batch
        optimizer.zero_grad()  # 이전 배치의 gradients를 초기화합니다.
        outputs = model(input_ids=input_ids, labels=labels)  # 모델에 입력하여 출력과 손실을 계산합니다.
        loss = outputs.loss  # 손실값 추출
        loss.backward()  # 역전파를 통해 모델 파라미터를 업데이트합니다.
        optimizer.step()  # 옵티마이저를 사용하여 모델 파라미터를 한 단계 업데이트합니다.
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')  # 현재 에포크와 손실값 출력

# 7. 모델 평가
# 학습이 완료된 모델을 평가 모드로 전환한 후, 새로운 데이터를 입력하여 요약 문장을 생성합니다.
# 장점: 평가 모드에서는 dropout 등 학습 시 사용되던 불확실성을 제거하여 안정적인 예측을 제공합니다.
# 단점: 생성된 요약 문장의 품질이 데이터셋의 특성에 크게 좌우될 수 있습니다.
model.eval()

# 새로운 데이터 예시로 요약 생성
new_texts = ["이것은 테스트를 위한 새로운 문장입니다."]
new_inputs = tokenizer(new_texts, max_length=512, truncation=True, return_tensors='pt', padding=True)

# 요약 문장 생성
with torch.no_grad():  # 평가 모드에서는 gradient 계산을 하지 않습니다.
    summary_ids = model.generate(new_inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
    summaries = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

print("Generated Summaries:", summaries)  # 생성된 요약 문장을 출력합니다.

# 8. 성능 평가
# Rouge 점수를 사용하여 모델이 생성한 요약 문장의 품질을 평가합니다.
# 장점: Rouge 점수는 요약 문장이 원본 문장을 얼마나 잘 반영하고 있는지를 정량적으로 평가할 수 있습니다.
# 단점: Rouge 점수는 길이가 긴 문장보다는 짧은 문장에 유리할 수 있습니다.
rouge = Rouge()
reference_summaries = ["이것은 테스트를 위한 요약 문장입니다."]  # 참고 요약 문장
scores = rouge.get_scores(summaries, reference_summaries)  # 생성된 요약과 참고 요약 비교

print("ROUGE Scores:", scores)  # Rouge 점수를 출력하여 성능을 평가합니다.
